In [ ]:
import os
import sys
import pandas as pd
import csv
import numpy as np
import statistics

In [ ]:
# READ DATASETS AND SAVE CSV DATA INTO PANDAS DATAFRAMES CASTING TIMESTAMPS

In [ ]:
df_batch_registry = pd.read_csv(r'dataset/batch_registry.csv', sep = ";", header = 0, index_col = False)
df_batch_registry

In [ ]:
df_cooking_metrics = pd.read_csv(r'dataset/cooking_metrics.csv', sep = ";", header = 0, index_col = False)
df_cooking_metrics['timestamp'] = pd.to_datetime(df_cooking_metrics['timestamp'])
df_cooking_metrics

In [ ]:
df_faulty_intervals = pd.read_csv(r'dataset/faulty_intervals.csv', sep = ";", header = 0, index_col = False)
df_faulty_intervals['start_time'] = pd.to_datetime(df_faulty_intervals['start_time'])
df_faulty_intervals['end_time'] = pd.to_datetime(df_faulty_intervals['end_time'])
df_faulty_intervals

In [ ]:
# JOIN METRICS AND BATCH REGISTRY

In [ ]:
df_metrics_join_arepas = pd.merge(df_cooking_metrics,df_batch_registry,on=['batch_id'])
df_metrics_join_arepas

In [ ]:
# SOME DATA PREPARATION IN ORDER TO GET FLOAT TYPE FOR METRICS

In [ ]:
df_metrics_join_arepas['metric_1'] = df_metrics_join_arepas['metric_1'].str.replace(',', '.')
df_metrics_join_arepas['metric_2'] = df_metrics_join_arepas['metric_2'].str.replace(',', '.')
df_metrics_join_arepas['metric_1'] = df_metrics_join_arepas['metric_1'].astype(float)
df_metrics_join_arepas['metric_2'] = df_metrics_join_arepas['metric_2'].astype(float)
df_metrics_join_arepas

In [ ]:
# FILTERING DATASETS FOR MACHINE AND AREPAS TYPE

In [ ]:
df_metrics_join_arepas_m1 = df_metrics_join_arepas.loc[df_metrics_join_arepas['machine_id'].str.contains('m1')]
df_metrics_join_arepas_m1_a1 = df_metrics_join_arepas_m1.loc[df_metrics_join_arepas_m1['arepa_type'].str.contains('a1')]
df_metrics_join_arepas_m1_a1

In [ ]:
df_faulty_intervals_m1 = df_faulty_intervals.loc[df_faulty_intervals['machine_id'].str.contains('m1')]
df_faulty_intervals_m1

In [ ]:
# CLEAN DATASET FROM FAULTY DATA

In [ ]:
for index_1, row_1 in df_metrics_join_arepas_m1_a1.iterrows():
    for index_2, row_2 in df_faulty_intervals_m1.iterrows():
        if(row_2['start_time']<row_1['timestamp']< row_2['end_time']):
            print(row_1['timestamp'])
            df_metrics_join_arepas_m1_a1 = df_metrics_join_arepas_m1_a1.drop(index_1)
            break
df_metrics_join_arepas_m1_a1

In [ ]:
# FUNCTION TO COMPUTE AVERAGE METRICS IN THE SPECIFIED INPUT TIMESTAMP
# THEN WRITE DATA IN CSV FILE IN ORDER TO GET NEW DATASET

In [ ]:
def compute_average(start_time_input, end_time_input):
    start_time = pd.to_datetime(start_time_input)
    end_time = pd.to_datetime(end_time_input)
    list_metric1 = []
    list_metric2 = []
    for index, row in df_metrics_join_arepas_m1_a1.iterrows():
        if(start_time<row['timestamp']< end_time):
            list_metric1.append(row['metric_1'])
            list_metric2.append(row['metric_2'])
    if(len(list_metric1)>0):
        avg_metric1 = statistics.mean(list_metric1)
        df_metrics_join_arepas_m1_a1['Avg_metric1'] = avg_metric1
        print('Average for metric1: ' + str(avg_metric1))
    else:
        print('No valid data for metric 1 in the specified interval')
    if(len(list_metric2)>0):
        avg_metric2 = statistics.mean(list_metric2)
        df_metrics_join_arepas_m1_a1['Avg_metric2'] = avg_metric2
        print('Average for metric2: ' + str(avg_metric2))
    else:
        print('No valid data for metric 2 in the specified interval')
    df_metrics_join_arepas_m1_a1['start_time'] = start_time
    df_metrics_join_arepas_m1_a1['end_time'] = end_time
    df_metrics_join_arepas_m1_a1.to_csv(r'dataset/average_metrics.csv', index=False)

In [ ]:
compute_average('2020-11-01T01:25:00','2020-11-01T01:36:00')

In [ ]:
# FUNCTION TO COMPUTE HOURLY AVERAGE METRICS IN THE SPECIFIED INPUT TIMESTAMP FOR MACHINE 1, KITCHEN 1, AREPA 1
# THEN WRITE DATA IN CSV FILE IN ORDER TO GET NEW DATASET

In [ ]:
def compute_hourly_average(start_time_input, end_time_input):
    start_time = pd.to_datetime(start_time_input)
    end_time = pd.to_datetime(end_time_input)
    valid_data = []
    for index, row in df_metrics_join_arepas_m1_a1.iterrows():
        if(start_time<row['timestamp']< end_time):
            valid_data.append(row)
    df_metrics_join_arepas_m1_a1_to_hourly = pd.DataFrame(valid_data)
    df_metrics_join_arepas_m1_a1_to_hourly['start_time'] = start_time
    df_metrics_join_arepas_m1_a1_to_hourly['end_time'] = end_time
    hour = pd.to_timedelta(df_metrics_join_arepas_m1_a1_to_hourly['timestamp'].dt.hour, unit='H')
    hour.name = 'Hour'
    df_hourly_average = df_metrics_join_arepas_m1_a1_to_hourly.groupby(hour).mean().reset_index()
    df_hourly_average['machine_id'] = 'm1'
    df_hourly_average['arepa_type'] = 'a1'
    df_hourly_average['Hour']
    df_hourly_average = df_hourly_average.drop(columns=['metric_1','metric_2'])
    df_hourly_average.to_csv(r'dataset/hourly_average_metrics.csv', index=False)
    return df_hourly_average

In [ ]:
# SAVE RESULTS ALSO IN DATAFRAME IN ORDER TO SHOW DATA AFTER CSV CREATION

In [ ]:
df_hourly_average = compute_hourly_average('2020-11-01T01:25:00','2020-11-01T01:36:00')

In [ ]:
df_hourly_average